# Small local agents

Experimenting with agents does not require access to external services, expensive hardware, or even the cloud.
This short tutorial notebook is a introduction to language model-based agents using small language models, and demonstrating their use to query a database.

# Setup

## Local Ollama server

The language models will we use will be served by Ollama.
The default instructions use `podman` to run that Ollama server in a rootless container.

This assumes that `podman` is installed. If not yet the case, the instructions are here:
https://podman.io/docs/installation

In [1]:
! podman run -d --rm -p 11434:11434 -v dot_ollama:/root/.ollama --name ollama docker.io/alpine/ollama

19a7f33e9100e1f667f371d6e5423223cdef8c889a2400943b5e7b78f5cdb707


## Required Python packages

The Python packages `ollama` and `langchain`, and `langchain_ollama` are required. Make sure they are installed before continuing.

In [2]:
import dataclasses
import langchain
import langchain_ollama
import ollama

## Utility code

In order to minimize code duplication, we encapsulate most of Ollama model management into a class.

In [3]:
@dataclasses.dataclass(frozen=True)
class OllamaModel:
    """Wrapper class for models with Ollama and LangChain."""

    name: str

    def __str__(self) -> str:
        return self.name

    def has_tools(self) -> bool:
        return 'tools' in self.show().capabilities

    def is_pulled(self) -> bool:
        for mdl in ollama.list().models:
            if mdl.model == self.name:
                return True
        return False

    def pull(self):
        """Pull the model to make it available to the local Ollama instance."""
        ollama.pull(self.name)

    def show(self) -> ollama._types.ShowResponse:
        return ollama.show(self.name)

    def new_chat(self, temperature:float = 0) -> langchain_ollama.ChatOllama:
        res = langchain_ollama.ChatOllama(
            model=str(self),
            temperature=temperature
        )
        return res

### Object display customization
We also create a convenience custom HTML render for AI responses.

In [4]:
import jinja2

_aimessage_template = jinja2.Template(    
    """<table>
    <thead><tr><td colspan="2">AI message</td></tr></thead>
    <tbody>
    <tr>
      {% if response.content %}
      <td style="vertical-align: top; border-right: 2px solid rgb(40, 40, 150); width: 6em;"><b>response:</b></td>
      <td style="text-align: left; text-indent: 0px;">
        <div style="white-space: pre-wrap;">
        {{ response.content }}
        </div>
      </td>
      {% endif %}
      {% if response.tool_calls %}
      <td style="vertical-align: top; border-right: 2px solid rgb(150, 40, 40); width: 6em;"><b>tools:</b></td>
      <td style="text-align: left; text-indent: 0px;">
        <div style="white-space: pre-wrap;">
        {{ response.tool_calls }}
        </div>
      </td>
      {% endif %}
  </tr></tbody></table>
  """)


def aimessage_html(obj):
    return _aimessage_template.render(response=obj)
    
html_formatter = get_ipython().display_formatter.formatters['text/html']
html_formatter.for_type_by_name('langchain_core.messages.ai', 'AIMessage', aimessage_html)

In [5]:
def toolmessage_html(obj):
    _toolmessage_template = jinja2.Template(    
        """<table>
        <thead><tr><td colspan="2">Tool message</td></tr></thead>
        <tbody>
        <tr>
          {% if response.content %}
          <td style="vertical-align: top; border-right: 2px solid rgb(40, 40, 150); width: 6em;"><b>response:</b></td>
          <td style="text-align: left; text-indent: 0px;">
            <div style="white-space: pre-wrap;">
            {{ response.content }}
            </div>
          </td>
          {% endif %}
        </tr>
        </tbody></table>
        """)
    return _toolmessage_template.render(response=obj)


html_formatter = get_ipython().display_formatter.formatters['text/html']
html_formatter.for_type_by_name('langchain_core.messages.tool', 'ToolMessage', toolmessage_html)

In [6]:
def highlight(text):
    return f'\n\033[1m{text}\033[0m'

## List of language models

Below is the list all models you want to use here. Be mindful about the capabilities of the system this will run on, in particular the RAM or the VRAM is a GPU is present.

This notebook was created with `gemma3:4b` (Gemma3 with 4B parameters) and `granite4:1b` (Granite4 with 1B parameters).

In [7]:
MODELS = (
    # OllamaModel('deepseek-r1:1.5b'),
    OllamaModel('gemma3:4b'),
    OllamaModel('granite4:1b'),
    # OllamaModel('phi4-mini:3.8b'),
    # OllamaModel('qwen3:4b')
)

We are ready to start. First, we ensure that the models we need are pulled locally.

In [8]:
for mdl in MODELS:
    if mdl.is_pulled():
        print(f'Model {mdl} already pulled.')
    else:
        print(f'Pulling model {mdl}...', end='', flush=True)
        mdl.pull()
        print('done.')

Model gemma3:4b already pulled.
Model granite4:1b already pulled.


# Language models and their limitations

In [9]:
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.system import SystemMessage
from langchain_core.tools import tool

In [10]:
human_message = HumanMessage('Make the following calculation: 392373366237 * 562310951975')
for mdl in MODELS:
    print(f'\n\033[1m{str(mdl)}\033[0m\n')
    chat = mdl.new_chat()
    response = chat.invoke([human_message])
    display(response)


gemma3:4b



AIMessage(content="Okay, let's calculate 392373366237 * 562310951975.\n\nThis is a large multiplication, and it's best done with a calculator or a programming language.  I'll use Python for this calculation:\n\n```python\nnum1 = 392373366237\nnum2 = 562310951975\n\nresult = num1 * num2\nprint(result)\n```\n\nThis code will output:\n\n2039999999999999999999999999999999", additional_kwargs={}, response_metadata={}, id='lc_run--019aff79-faa6-7cd0-8dd5-32dc773eabdd-0')


granite4:1b



AIMessage(content='The result of multiplying 392373366237 by 562310951975 is:\n\n220,646,204,278,964,000,795', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-08T19:40:50.682884839Z', 'done': True, 'done_reason': 'stop', 'total_duration': 41723327372, 'load_duration': 3010227073, 'prompt_eval_count': 46, 'prompt_eval_duration': 18709831723, 'eval_count': 31, 'eval_duration': 19975427110, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019aff7a-7d7b-7f71-a5f4-00c2bfe87062-0', usage_metadata={'input_tokens': 46, 'output_tokens': 31, 'total_tokens': 77})


Is this correct? The actual answer is:

In [11]:
print(f'{392373366237 * 562310951975:,}')

220,635,841,098,362,793,468,075


Unless this notebook is running in a future when LLMs have learned to multiply, or this notebook became part of the training data, the LLM answer is likely not correct.

The point to remember is to not rely on the current LLM technology to perform correctly mathematical operations, or more generally to evaluate expressions or run algorithms. However, they might be able to generate code (software instructions), for example a function call, to get to the answer. This can be observed when using `gemma3:4b`: the answer from the LLM shows includes Python code that returns the correct answer is evaluated.

If we have a way to know when LLM answers contains code, we could simply evaluate that code and consider the resulting text its answer a dialogue. In fact, it does not even have to be a dialogue and code evaluation represent simply an action.

Giving additional instructions in the prompt can get us there with this example. We can add that if the pattern matching in the LLMs considers that evaluating code would be a better path to the correct answer, only code should be returned. To facilitate the identification of such answer we also indicate the format to use.

In [12]:
answer_as_code = HumanMessage("""
IF this type of question would be better answered by evaluating code, return instead
ONLY Python code in the answer. Use the Markdown format for code snippets:
```python
<code>
```
""")
human_message = HumanMessage("""Make the following calculation: 392373366237 * 562310951975""")
for mdl in MODELS:
    print(highlight(str(mdl)))
    chat = mdl.new_chat()
    response = chat.invoke([human_message, answer_as_code])
    display(response)


gemma3:4b


AIMessage(content='```python\n392373366237 * 562310951975\n```', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-12-08T19:40:58.561632809Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7847118974, 'load_duration': 277482052, 'prompt_eval_count': 83, 'prompt_eval_duration': 1363682037, 'eval_count': 32, 'eval_duration': 6120723632, 'logprobs': None, 'model_name': 'gemma3:4b', 'model_provider': 'ollama'}, id='lc_run--019aff7b-2099-75a0-bc6b-96a1a245b304-0', usage_metadata={'input_tokens': 83, 'output_tokens': 32, 'total_tokens': 115})


granite4:1b


AIMessage(content='```python\nresult = 392373366237 * 562310951975\nresult\n```', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-08T19:41:13.720214429Z', 'done': True, 'done_reason': 'stop', 'total_duration': 15130838249, 'load_duration': 124583662, 'prompt_eval_count': 86, 'prompt_eval_duration': 11476726492, 'eval_count': 21, 'eval_duration': 3507454484, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019aff7b-3f5b-77e2-95bb-182ac411f156-0', usage_metadata={'input_tokens': 86, 'output_tokens': 21, 'total_tokens': 107})

We check our new prompt giving the option to answer with code with a question that should be answerable without code.
That is a question for which the answer was almost certainly in the training data for the model.

In [13]:
human_message = HumanMessage("""What is the name of the natural satellite orbiting around the earth?""")
for mdl in MODELS:
    print(highlight(str(mdl)))
    chat = mdl.new_chat()
    response = chat.invoke([human_message, answer_as_code])
    display(response)


gemma3:4b


AIMessage(content='```python\nprint("Moon")\n```', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-12-08T19:41:17.283402346Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3537444852, 'load_duration': 318737561, 'prompt_eval_count': 64, 'prompt_eval_duration': 1770844038, 'eval_count': 10, 'eval_duration': 1421325110, 'logprobs': None, 'model_name': 'gemma3:4b', 'model_provider': 'ollama'}, id='lc_run--019aff7b-7a90-76a1-9cd5-15358a03b683-0', usage_metadata={'input_tokens': 64, 'output_tokens': 10, 'total_tokens': 74})


granite4:1b


AIMessage(content='The name of the natural satellite orbiting around the Earth is called **the Moon**.', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-08T19:41:30.022761765Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12713884135, 'load_duration': 110351251, 'prompt_eval_count': 83, 'prompt_eval_duration': 8633197229, 'eval_count': 19, 'eval_duration': 3948868046, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019aff7b-887a-7cb1-a5e9-4558ab6c99eb-0', usage_metadata={'input_tokens': 83, 'output_tokens': 19, 'total_tokens': 102})

While `gemma3:4b` its technically correct, it may indicate that this model will resort to writing code more often than necessary.

# Adding tools as context

Agents are language models to which the availability of tools (software functions), with a description of what they do, is specified as context. For `gemma3:4b` this can be achieved through a system prompt, while a few other models with a "tools" capability Python functions can passed directly. This is the case with `granite4:1b` for example.

## Tools in prompt

We showed earlier that a prompt can indicate that code evaluation is available.
Similarly, we can indicate that tools are available and calling a tool can be used as an answer.

If we make our multiplication a tool, a prompt can look like follows:

In [14]:
system_prompt_tools = """
You have access to functions. 
You don't have to use tools if not necessary, but *IF* you decide to invoke any of the function(s),
you MUST put it in a valid JSON expression like:
[
  {"name": function name,
   "parameters": dictionary of argument name and its value}
]

You SHOULD NOT include any other text in the response if you call a function.

Functions:
[
  { "name": "multiply",
    "description": "Multiply two numbers.",
    "parameters": {
      "type": "object",
      "properties": {
          "a": "number",
          "b": "number"
      },
      "required": [
          "a", "b"
      ]
    }
  }
]
"""

## Tools as `tools`

The use of tools has been somewhat formalized since the early days of prompt engineering, a mere double-digit number
of months in the past, we have seen so far.

Many of the most recent language model specifically include a capability called "tools". This frees a user, or developer,
from having to work on prompt sections that declare the availablity of tools in a way that the language model reacts to it, and on parsing out if an answer is text or code. `langchain` provides a wrapper for this and let one simply declare any Python function as tool. For the multiplication we can just plug the Python multiplication operator.

In [15]:
import operator
tools = [tool(operator.mul)]

## Language model opting to answer with a tool call

We have shown that a prompt context, wrapped in a `langchain` tool declaration when the model has the capability, can lead a language model to answer with code. If we ask again our multiplication question:

In [16]:
human_message = HumanMessage("""Make the following calculation: 392373366237 * 562310951975""")

for model in MODELS:
    chat = model.new_chat()
    if model.has_tools():
        print(f'{highlight(model.name)} (with tools)\n')
        chat = chat.bind_tools(tools)
        messages = [human_message]
    else:
        print(f'{highlight(model.name)}\n')
        messages = [SystemMessage(system_prompt_tools),
                    human_message]
    response = chat.invoke(messages)
    display(response)


gemma3:4b



AIMessage(content='```json\n[\n  {\n    "name": "multiply",\n    "parameters": {\n      "a": 392373366237,\n      "b": 562310951975\n    }\n  }\n]\n```', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-12-08T19:42:02.454446058Z', 'done': True, 'done_reason': 'stop', 'total_duration': 32194077932, 'load_duration': 335894417, 'prompt_eval_count': 224, 'prompt_eval_duration': 5376817252, 'eval_count': 69, 'eval_duration': 26433051974, 'logprobs': None, 'model_name': 'gemma3:4b', 'model_provider': 'ollama'}, id='lc_run--019aff7b-bb11-7e61-82c4-b89d7aef939e-0', usage_metadata={'input_tokens': 224, 'output_tokens': 69, 'total_tokens': 293})


granite4:1b (with tools)



AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-08T19:42:18.42391671Z', 'done': True, 'done_reason': 'stop', 'total_duration': 15884430814, 'load_duration': 89324640, 'prompt_eval_count': 199, 'prompt_eval_duration': 11520595603, 'eval_count': 31, 'eval_duration': 4243729556, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019aff7c-3929-7f01-ae3c-d46f83b4d00a-0', tool_calls=[{'name': 'mul', 'args': {'a': '392373366237', 'b': '562310951975'}, 'id': 'e5b6cb9c-bb6e-42ac-86a6-aff3fc9dda05', 'type': 'tool_call'}], usage_metadata={'input_tokens': 199, 'output_tokens': 31, 'total_tokens': 230})

# Agentic linear chaining - an example

## Database

In [17]:
import sqlite3

dbcon = sqlite3.connect(':memory:', check_same_thread=False)
dbcon.execute("""
CREATE TABLE medication(
ndc STRING PRIMARY KEY,
common_name STRING,
quantity INTEGER
);
""")
for values in (
    ('ABCD-DEF', 'Aspirin', 3), ('ABCD-GHI', 'Aspirin', 5), ('GHIK-KLM', 'bacitracin', 10)
):
    dbcon.execute("""
    INSERT INTO medication(ndc, common_name, quantity) VALUES (?,?,?);
    """, values)



Functions to query the database:
- `get_ndc_code` to get all NDC codes for the common name of a medication
- `get_medication_stock` to get the stock for a given NDC code

In [18]:
def get_ndc_code(medication: str) -> tuple[str, ...]:
    """Get possible ndc codes for a medication.
    
    Args:
      medication: name of the medication for which NDC code should be returned.
    """
    cursor = dbcon.cursor()
    cursor.execute('SELECT ndc FROM medication WHERE common_name==?',
                   (medication, ))
    return tuple(row[0] for row in cursor.fetchall())


def get_ndc_stock(ndc_code: str) -> int:
    """Get the number of boxes of medication in stock.
    
    Args:
      ndc_code: NDC code to retrieve stock for.
    """
    cursor = dbcon.cursor()
    cursor.execute('SELECT quantity FROM medication WHERE ndc==?',
                   (ndc_code, ))
    return cursor.fetchone()[0]

The prompt may contain several tools available. A prompt context will be similar to the one we showed for the multiplication. 

In [19]:
system_prompt_tools = """
You have access to functions. 
You don't have to use tools if not necessary, but *IF* you decide to invoke any of the function(s),
you MUST put it in a valid JSON expression like:
[
  {"name": function name,
   "parameters": dictionary of argument name and its value}
]

You SHOULD NOT include any other text in the response if you call a function.

Functions:
[
  {
    "name": "get_ndc_stock",
    "description": "Get the number of boxes of medication in stock.",
    "parameters": {
      "type": "object",
      "properties": {
        "ndc_code": {
          "type": "string"
        }
      },
      "required": [
        "ndc_code"
      ]
    }
  },
  {
    "name": "get_ndc_code",
    "description": "Get possible ndc codes for a medication.",
    "parameters": {
      "type": "object",
      "properties": {
        "medication": {
          "type": "string"
        }
      },
      "required": [
        "medication"
      ]
    },
  }
]
"""

Whenever a model has the capabilities "tools", the user or developer is spared the prompt context engineering.
The decorator `tool()` in `langchain` can be used instead.

Note: `langchain` is essentially an abstraction layer over prompts.
It can create the equivalent of our engineered prompt contexts from Python function definitions,
feeding the LLM with information in the docstring, or in additional declarative structures
(see https://docs.langchain.com/oss/python/langchain/tools#advanced-schema-definition).

In [20]:
tools = [tool(get_ndc_code), tool(get_ndc_stock)]

In [21]:
def invoke_model(model, human_message, system_message=SystemMessage('')):
    chat = model.new_chat()
    if model.has_tools():
        print(f'\n{highlight(model.name)} (with tools)\n')
        chat = chat.bind_tools(tools)
    else:
        print(f'\n{highlight(model.name)}\n')
    messages = [system_message,
                human_message]
    response = chat.invoke(messages)
    return response, chat

human_message = HumanMessage('Name the first month of the year.')

for model in MODELS:
    response, chat = invoke_model(model, human_message)
    display(response)



gemma3:4b



AIMessage(content='January! 😊 \n\nDo you want to play a quick game of guessing games?', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-12-08T19:42:24.94549934Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6288765459, 'load_duration': 279177011, 'prompt_eval_count': 22, 'prompt_eval_duration': 628894393, 'eval_count': 18, 'eval_duration': 5363950493, 'logprobs': None, 'model_name': 'gemma3:4b', 'model_provider': 'ollama'}, id='lc_run--019aff7c-781f-7b53-b224-0d6959735615-0', usage_metadata={'input_tokens': 22, 'output_tokens': 18, 'total_tokens': 40})



granite4:1b (with tools)



AIMessage(content='The first month of the year is **January**.', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-08T19:42:42.398600321Z', 'done': True, 'done_reason': 'stop', 'total_duration': 17347730278, 'load_duration': 82920434, 'prompt_eval_count': 268, 'prompt_eval_duration': 15096003132, 'eval_count': 12, 'eval_duration': 2147303290, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019aff7c-9118-70e2-bf33-034673ccaf83-0', usage_metadata={'input_tokens': 268, 'output_tokens': 12, 'total_tokens': 280})

In [22]:
tool_map = {tool.name: tool for tool in tools}
if len(tool_map) != len(tools):
    raise ValueError("'tools' contains at lease one tool name duplicate.")

In [23]:
human_message = HumanMessage('How many boxes of medication with NDC code ABCD-DEF do we have left?')

for model in MODELS:
    response, chat = invoke_model(model, human_message, SystemMessage(system_prompt_tools))
    display(response)
    if response.tool_calls:
        for tool_call in response.tool_calls:
            print(
                f'  --> calling {tool_call["name"]}'
                f'({", ".join("=".join((k, repr(v))) for k, v in tool_call["args"].items())})'
            )
            selected_tool = tool_map[tool_call['name']]
            res = selected_tool.invoke(tool_call)
            print('      result:')
            print(f'        {res.content}')
    else:
        print('Assessing automatically if there is a tool call is left as an exercise for the reader.')



gemma3:4b



AIMessage(content='```json\n[\n  {\n    "name": "get_ndc_stock",\n    "parameters": {\n      "ndc_code": "ABCD-DEF"\n    }\n  }\n]\n```', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-12-08T19:43:14.71736512Z', 'done': True, 'done_reason': 'stop', 'total_duration': 32130731345, 'load_duration': 293680086, 'prompt_eval_count': 318, 'prompt_eval_duration': 7484939469, 'eval_count': 50, 'eval_duration': 24301447787, 'logprobs': None, 'model_name': 'gemma3:4b', 'model_provider': 'ollama'}, id='lc_run--019aff7c-d599-72f2-a604-e80b3480a772-0', usage_metadata={'input_tokens': 318, 'output_tokens': 50, 'total_tokens': 368})

Assessing automatically if there is a tool call is left as an exercise for the reader.


granite4:1b (with tools)



AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-08T19:43:37.034360037Z', 'done': True, 'done_reason': 'stop', 'total_duration': 22132885904, 'load_duration': 196241314, 'prompt_eval_count': 518, 'prompt_eval_duration': 16952269961, 'eval_count': 27, 'eval_duration': 4939081527, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019aff7d-53c9-7881-bf4f-5e6fef94f55a-0', tool_calls=[{'name': 'get_ndc_stock', 'args': {'ndc_code': 'ABCD-DEF'}, 'id': '2b26efbe-2735-4f51-89dc-2f2f6647b2ff', 'type': 'tool_call'}], usage_metadata={'input_tokens': 518, 'output_tokens': 27, 'total_tokens': 545})

  --> calling get_ndc_stock(ndc_code='ABCD-DEF')
      result:
        3


In [24]:
human_message = HumanMessage("What are possible NDC codes for the medication 'Aspirin'?")

for model in MODELS:
    response, chat = invoke_model(model, human_message, SystemMessage(system_prompt_tools))
    display(response)
    if response.tool_calls:
        print('      result:')
        for tool_call in response.tool_calls:
            selected_tool = tool_map[tool_call['name']]
            res = selected_tool.invoke(tool_call)    
            print(f'        {res.content}')
    else:
        print('Assessing automatically if there is a tool call is left as an exercise for the reader.')



gemma3:4b



AIMessage(content='```json\n[\n  {\n    "name": "get_ndc_code",\n    "parameters": {\n      "medication": "Aspirin"\n    }\n  }\n]\n```', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-12-08T19:43:47.740582126Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10545446906, 'load_duration': 296672173, 'prompt_eval_count': 315, 'prompt_eval_duration': 1035159792, 'eval_count': 48, 'eval_duration': 9154730895, 'logprobs': None, 'model_name': 'gemma3:4b', 'model_provider': 'ollama'}, id='lc_run--019aff7d-aae9-7b73-95c0-1deef2a64f24-0', usage_metadata={'input_tokens': 315, 'output_tokens': 48, 'total_tokens': 363})

Assessing automatically if there is a tool call is left as an exercise for the reader.


granite4:1b (with tools)



AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-08T19:44:02.318015134Z', 'done': True, 'done_reason': 'stop', 'total_duration': 14463488728, 'load_duration': 133969511, 'prompt_eval_count': 514, 'prompt_eval_duration': 9727646180, 'eval_count': 25, 'eval_duration': 4568147270, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019aff7d-d48c-7113-8c5b-4ada27a19d71-0', tool_calls=[{'name': 'get_ndc_code', 'args': {'medication': 'Aspirin'}, 'id': 'c072a779-c583-4add-b4e2-0dad061774a1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 514, 'output_tokens': 25, 'total_tokens': 539})

      result:
        ["ABCD-DEF", "ABCD-GHI"]


Cool. Uut what did we exactly gain here? If you are are programmer or a data analyst knowing how to write SQL, arguably not much. However, what we have achieved is akin to a text user interface, or language user interface, that can be an alternative to a graphical user interface.

## Reasoning in steps

So far we only showed questions for which either the language model had the answer memorized from the data it was trained on, or for which the answer was one tool call away.

There are questions questions for which the answer requires a sequence of recall from training data or tool calls, with the output of a step used as the input to the next step. For example, asking for how many Aspirin boxes are in stock, would require to 1) get the NDC codes for Aspirin products, 2) query the stock for each, and finally 3) sum those values.

In [25]:
human_message = HumanMessage('How many boxes of Aspirin do we have in stock altogether?')

for model in MODELS:
    response, chat = invoke_model(model, human_message, SystemMessage(system_prompt_tools))
    display(response)



gemma3:4b



AIMessage(content='```json\n[\n  {\n    "name": "get_ndc_code",\n    "parameters": {\n      "medication": "Aspirin"\n    }\n  },\n  {\n    "name": "get_ndc_stock",\n    "parameters": {\n      "ndc_code": "0003499"\n    }\n  }\n]\n```', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-12-08T19:44:31.516590916Z', 'done': True, 'done_reason': 'stop', 'total_duration': 29078289523, 'load_duration': 321380361, 'prompt_eval_count': 315, 'prompt_eval_duration': 744411909, 'eval_count': 93, 'eval_duration': 27906190738, 'logprobs': None, 'model_name': 'gemma3:4b', 'model_provider': 'ollama'}, id='lc_run--019aff7e-0d84-7142-a6ec-c5960b68aef4-0', usage_metadata={'input_tokens': 315, 'output_tokens': 93, 'total_tokens': 408})



granite4:1b (with tools)



AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-08T19:44:44.407637549Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12814503591, 'load_duration': 97660305, 'prompt_eval_count': 514, 'prompt_eval_duration': 9037216110, 'eval_count': 25, 'eval_duration': 3647147547, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019aff7e-7f67-7ed0-aed5-6bc8f2d37a53-0', tool_calls=[{'name': 'get_ndc_code', 'args': {'medication': 'Aspirin'}, 'id': '2d0c4128-8708-407d-9f35-12332528adc6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 514, 'output_tokens': 25, 'total_tokens': 539})

Odds are are that the results are not great. Bare language models used in this notebook generally do not handle this well without guidance from a prompt. For example,  

## ReAct

The ReAct approach prompts a language model to match its answers to the following pattern:
1. **question**
2. **thought:** the language model's answer to the question. The answer can be the use of a tool.
3. **action:** whenever the answer is the use of a tool, a function call, this is selected here.
4. **action input:** the input to the action.
5. **observation:** observe the result of performing the action.
6. The sequence of steps 2 to 5 can be repeated N times
7. **final answer**

It is derived from the empirical observation that LLMs appeared to give more "thoughtful" answers, or better solve reasoning-oriented questions when asked to think step by step.

React can be idependently achieved through adding the description to a prompt and iteratively parsing language model responses and crafting new messages. However, in many scenario this is can be abstracted out through the use of library. We use `langchain` again to demonstrate how it can work:

In [26]:
from langchain.agents import create_agent

human_message = HumanMessage("How many boxes of Aspirin do we have in stock altogether?")

for model in MODELS:
    print(highlight(model.name))
    if model.has_tools():
        react = create_agent(model.new_chat(), tools=tools)
        for chunk in react.stream({'messages': [human_message]}, stream_mode='updates'):
            for k in chunk.keys():
                for m in chunk[k]['messages']:
                    display(m)
    else:
        print('Parsing AI messages into next actions and results is left as an exercise.')


gemma3:4b
Parsing AI messages into next actions and results is left as an exercise.

granite4:1b


AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-08T19:44:55.966593969Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11319965889, 'load_duration': 92762577, 'prompt_eval_count': 274, 'prompt_eval_duration': 4862770074, 'eval_count': 25, 'eval_duration': 6339518881, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019aff7e-b265-7b62-9b54-136424e513a7-0', tool_calls=[{'name': 'get_ndc_code', 'args': {'medication': 'Aspirin'}, 'id': '5431dcce-5fd8-439c-bddc-5eee0c2ce893', 'type': 'tool_call'}], usage_metadata={'input_tokens': 274, 'output_tokens': 25, 'total_tokens': 299})

ToolMessage(content='["ABCD-DEF", "ABCD-GHI"]', name='get_ndc_code', id='a6d16352-6aec-4662-be37-1e067b9ee01a', tool_call_id='5431dcce-5fd8-439c-bddc-5eee0c2ce893')

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-08T19:45:01.73438538Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5752588051, 'load_duration': 50907407, 'prompt_eval_count': 323, 'prompt_eval_duration': 1113221688, 'eval_count': 27, 'eval_duration': 4562128881, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019aff7e-deac-7ed1-9c16-fbe4db80ac77-0', tool_calls=[{'name': 'get_ndc_stock', 'args': {'ndc_code': 'ABCD-DEF'}, 'id': '809d80b5-82ca-4e67-b38c-8ef1f8f8f5ed', 'type': 'tool_call'}], usage_metadata={'input_tokens': 323, 'output_tokens': 27, 'total_tokens': 350})

ToolMessage(content='3', name='get_ndc_stock', id='cef8e30b-29a4-4f95-94e7-58c7281ca124', tool_call_id='809d80b5-82ca-4e67-b38c-8ef1f8f8f5ed')

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-08T19:45:07.050301472Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5306337147, 'load_duration': 74687734, 'prompt_eval_count': 364, 'prompt_eval_duration': 721335696, 'eval_count': 26, 'eval_duration': 4481270826, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019aff7e-f52f-7453-839f-1efbb1bed773-0', tool_calls=[{'name': 'get_ndc_stock', 'args': {'ndc_code': 'ABCD-GHI'}, 'id': 'd79c5738-7b77-4aa9-9dac-860211704613', 'type': 'tool_call'}], usage_metadata={'input_tokens': 364, 'output_tokens': 26, 'total_tokens': 390})

ToolMessage(content='5', name='get_ndc_stock', id='61e6d106-f5ac-415f-8468-d3991c30d716', tool_call_id='d79c5738-7b77-4aa9-9dac-860211704613')

AIMessage(content='The total number of boxes in stock for Aspirin is **8**.', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-08T19:45:10.836298484Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3772480912, 'load_duration': 69314122, 'prompt_eval_count': 405, 'prompt_eval_duration': 453153081, 'eval_count': 17, 'eval_duration': 3226539227, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019aff7f-09f5-7960-ba03-6341d65df4cf-0', usage_metadata={'input_tokens': 405, 'output_tokens': 17, 'total_tokens': 422})

The answer is indeed correct.

# Conclusion

Small language models are able to handle simple queries and chain sequences of steps to reach an answer. They can run locally / isolated and on relatively modest hardware.